## 파일 불러오기
학습 데이터, 테스트 데이터 로딩

In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf) # print all numpy values
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder, MultiLabelBinarizer

In [2]:
categorical = ['주야', '요일', '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', 
            '도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류']
numerical = ['사상자수', '사망자수', '중상자수', '경상자수','부상신고자수']

x_train_num = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=numerical)

x_train_cat = pd.read_csv('./교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', usecols=categorical)


In [3]:
x_test_num = pd.read_csv('./test_kor.csv',encoding='cp949', usecols=numerical)

x_test_cat = pd.read_csv('./test_kor.csv',encoding='cp949', usecols=categorical)

## One Hot Encdoing

In [4]:
all_data = pd.concat((x_test_cat.dropna(), x_train_cat))
# for col in all_data.select_dtypes(include=[np.object]).columns:
#     print(col, all_data[col].unique())

In [5]:
for column in all_data.select_dtypes(include=[np.object]).columns:
    x_train_cat[column] = x_train_cat[column].astype('category', categories = all_data[column].unique())
    x_test_cat[column] = x_test_cat[column].astype('category', categories = all_data[column].unique())

/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  
/home/iron/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
x_train_cat = pd.get_dummies(data=x_train_cat)
x_test_cat = pd.get_dummies(data=x_test_cat)

In [7]:
# One Encoding Shape 확인
print(x_train_cat.shape)
print(x_test_cat.shape)

(25037, 328)
(50, 328)


# Deep Learning

In [8]:
import keras
from keras.layers import Dense, Input, LSTM, concatenate, Dropout, Conv2D, MaxPool2D, Embedding, Reshape, Conv1D
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import metrics
from keras import backend as K

Using TensorFlow backend.


In [9]:
# Train Data Shape
print(x_train_cat.shape)
print(x_train_num.shape)
# Test Data Shape
print(x_test_cat.shape)
print(x_test_num.shape)

(25037, 328)
(25037, 5)
(50, 328)
(50, 5)


In [10]:
# Numeric cases
Case1 = ['사망자수','사상자수','경상자수']
Case2 = ['사상자수', '중상자수', '부상신고자수']
Case3 = ['사상자수', '중상자수', '경상자수' ]
Case4 = ['사망자수', '사상자수', '중상자수' ]

# Categorical cases
Case5 = ['사고유형_대분류', '사고유형_중분류', '법규위반']
Case6 = ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류']
Case7 = ['도로형태_대분류', '도로형태', '당사자종별_2당_대분류']
Case8 = ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류']
Case11 = ['발생지시도', '발생지시군구']
Case12 = ['요일', '사고유형_대분류', '사고유형_중분류']
Case13 = ['요일', '사고유형_중분류', '법규위반', '도로형태_대분류']

# Mixed cases
Case9 = ['사망자수', '사상자수', '발생지시군구']
Case10 = ['중상자수', '경상자수', '발생지시군구']
Case14 = ['사망자수', '사상자수', '주야', '당사자종별_1당_대분류']
Case15 = ['사상자수', '중상자수', '주야', '도로형태']

Cases = [Case1, Case2, Case3, Case4, Case5, Case6, Case7, Case8, Case9, Case10,
         Case11, Case12, Case13, Case14, Case15]

In [11]:
# case ranges

case1_range = [0, 1]
case2_range = [2, 3]
case3_range = [4, 6]
case4_range = [7, 9]
case5_range = [10, 19]
case6_range = [20, 22]
case7_range = [23, 25]
case8_range = [26, 29]
case9_range = [30, 31]
case10_range = [32, 34]
case11_range = [35, 39]
case12_range = [40, 41]
case13_range = [42, 44]
case14_range = [45, 46]
case15_range = [47, 49]

test_ranges = [case1_range, case2_range, case3_range, case4_range, case5_range, case6_range,
             case7_range, case8_range, case9_range, case10_range, case11_range, case12_range,
             case13_range, case14_range, case15_range]

In [12]:
# train, set result in new array
for (idx, case), range_x in zip(enumerate(Cases), test_ranges):
    print(idx+1, case, range_x)

1 ['사망자수', '사상자수', '경상자수'] [0, 1]
2 ['사상자수', '중상자수', '부상신고자수'] [2, 3]
3 ['사상자수', '중상자수', '경상자수'] [4, 6]
4 ['사망자수', '사상자수', '중상자수'] [7, 9]
5 ['사고유형_대분류', '사고유형_중분류', '법규위반'] [10, 19]
6 ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류'] [20, 22]
7 ['도로형태_대분류', '도로형태', '당사자종별_2당_대분류'] [23, 25]
8 ['도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류'] [26, 29]
9 ['사망자수', '사상자수', '발생지시군구'] [30, 31]
10 ['중상자수', '경상자수', '발생지시군구'] [32, 34]
11 ['발생지시도', '발생지시군구'] [35, 39]
12 ['요일', '사고유형_대분류', '사고유형_중분류'] [40, 41]
13 ['요일', '사고유형_중분류', '법규위반', '도로형태_대분류'] [42, 44]
14 ['사망자수', '사상자수', '주야', '당사자종별_1당_대분류'] [45, 46]
15 ['사상자수', '중상자수', '주야', '도로형태'] [47, 49]


In [13]:
def save_result():
    '''
    test file에 있는 값을 result file에 저장
    '''
    test_file = pd.read_csv('./test_kor_1.csv', encoding='cp949', names= [chr(y) for y in range(ord('A'),ord('P')+1)])
    result_file = pd.read_csv('./result_kor.csv', encoding='cp949')
    
    cols = result_file['열'].values
    rows = result_file['행'].values
    vals = result_file['값'].astype('str').values
    
    rows-=1
    
    for i, (row, col) in enumerate(zip(rows, cols)):      
        vals[i] = test_file[col][row]

    rows += 1

    with open('./result_kor1.csv', 'wb') as f:
        for i in result_file.index:
            np.savetxt(f,  np.c_[rows,cols,vals],  delimiter=",", fmt='%s', encoding='cp949', header='행,열,값', comments='')
    print('Save Success')

In [14]:
# 수치형 데이터 Case 함수

def numeric_case(case, start, end):
    
    K.clear_session()
    case_copy=case.copy()
    
    # Case 확인
    print("Case:", case)
    
    # Train Data
    X = x_train_num.drop(columns=case)
    X = pd.concat([X, x_train_cat], axis=1).values
    
    # Test Data
    X_test = x_test_num.drop(columns=case)
    X_test = pd.concat([X_test, x_test_cat],axis=1).values
    
    # Label Data
    if '사상자수' in case:       
        case_copy.remove('사상자수')
        print('사상자제거:', case_copy) 
    Y = x_train_num[case_copy].values
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    # Model define

    num_input = Input(shape=( X.shape[1], 1), name='num_input')
    x = LSTM(512)(num_input)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y[0]), name='num_output')(x)
    
    
    # Model define
#     num_input = Input(shape=(len(X[0]),), name='num_input')
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(num_input)
#     x = Dropout(0.3)(x)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense((int(len(X[0] + len(Y[0])) * 2 /3)), activation='relu')(x)
#     x = Dropout(0.3)(x)
#     num_output = Dense(len(Y[0]), name='num_output')(x)

    model = Model(inputs=num_input, outputs=num_output)

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=25, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15)# val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]
    
    history = model.fit(X, Y, epochs=50, batch_size=128, callbacks=callbacks, validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    
    del model
    
    return Y_test

In [15]:
# 범주형 데이터 Case 함수

def categorical_case(case, start, end):
    
    K.clear_session()
    
    # Case 확인
    print("Case:", case)
    
    col_name = [] # ex. '사고유형_대분류_차대차', '사고유형_대분류_차대사람', '사고유형_대분류_차량단독'
    label_name = [] #  ex. '차대차', '차대사람', '차량단독
    
    # One Hot Encoding 후 Columns 이름과 Columns에 들어 있는 값 
    for col in case:
        label_name.extend(all_data[col].unique()) 
        for name in all_data[col].unique():
            col_name.append(col+'_'+name)

    # Train Data 
    X = x_train_cat.drop(columns=col_name)
    X = pd.concat([X, x_train_num], axis=1).values

    # Test Data
    X_test = x_test_cat.drop(columns=col_name)
    X_test = pd.concat([X_test, x_test_num],axis=1).values
    
    # Label Data
    Y = x_train_cat[col_name].values
    
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    
    cat_input = Input(shape=( X.shape[1], 1), name='cat_input')
    x = LSTM(512)(cat_input)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    cat_output = Dense(len(Y[0]), activation='sigmoid', name='cat_output')(x)
    
    # Model define
#     cat_input = Input(shape=(len(X[0]),), name='cat_input')
#     x = Dense(512, activation='relu')(cat_input)
#     x = Dropout(0.3)(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     cat_output = Dense(len(Y[0]), activation='sigmoid', name='cat_output')(x)

    model = Model(inputs=cat_input, outputs=cat_output)

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=25, verbose=1, factor=0.5, min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=15), # val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
        ]

    history = model.fit(X, Y, epochs=50, batch_size=128, callbacks=callbacks,validation_split=0.2 )
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])
    

    '''
    예시 출력:
        사고유형_대분류 : 차량단독
        사고유형_중분류 : 공작물충돌
        법규위반 : 안전운전 의무 불이행
    '''
    result = []
    for cat in Y_test: 
        x_list = list(cat)
        label_name_x = label_name.copy()
        temp = []
        for col in case:
#             print(col, ':', label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))] )
            temp.append(label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))])
            del x_list[:len(all_data[col].unique())]
            del label_name_x[:len(all_data[col].unique())]
        result.append(temp)
        
        
    del model
    
    return np.array(result)


In [16]:
# 믹스형 데이터 Case 함수

def mix_case(case, n, start, end):
    '''
    case: Case에 해당되는 컬럼이 담긴 배열
    n: 범주형 데이터 수
    start: 해당 Case 테스트의 시작 인덱스
    end: 해당 Case 테스트의 마지막 인덱스
    '''
    
    case_copy=case.copy()
    
    # categorical
    col_name = []
    label_name = []
    cat_name = case_copy[-n:]
    
    for col in case_copy[-n:]:
        label_name.extend(all_data[col].unique()) 
        for name in all_data[col].unique():
            col_name.append(col+'_'+name)
    
    Y_cat = x_train_cat[col_name].values
    X1 = x_train_cat.drop(columns=col_name)
    X_test1 = x_test_cat.drop(columns=col_name)
    
    # categorical columns 삭제
    del case_copy[-n:]
    
    # numerical
    X2 = x_train_num.drop(columns=case_copy)
    X_test2 = x_test_num.drop(columns=case_copy)
    
    if '사상자수' in case: 
        case_copy.remove('사상자수')
    Y_num = x_train_num[case_copy].values
    
    X = pd.concat([X1, X2], axis=1).values
    X_test = pd.concat([X_test1, X_test2],axis=1).values
    
    
    # Model define
    cat_input = Input(shape=(len(X[0]),), name='cat_input')
    x = Dense(1024, activation='relu')(cat_input)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    if n == 1:
        cat_output = Dense(len(Y_cat[0]), activation='softmax', name='cat_output')(x)
    elif n == 2:
        cat_output = Dense(len(Y_cat[0]), activation='sigmoid', name='cat_output')(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    num_output = Dense(len(Y_num[0]), name='num_output')(x)

    model = Model(inputs=cat_input, outputs=[cat_output, num_output])

    if n==1:
        model.compile(optimizer='adam',
                      loss={'cat_output': 'categorical_crossentropy', 'num_output': 'mse'},
                      metrics=['accuracy'])
    elif n==2:
        model.compile(optimizer='adam',
                      loss={'cat_output': 'binary_crossentropy', 'num_output': 'mse'},
                      metrics=['accuracy'])
    
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='cat_output_acc', 
                                            patience=25, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00000001)

    callbacks = [
        learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
        EarlyStopping('val_loss', patience=20), # val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
    ]

    history = model.fit(X, {'cat_output':Y_cat, 'num_output':Y_num}, epochs=50, batch_size=128, callbacks=callbacks,validation_split=0.2 )
    
    
    # make a prediction
    Y_test = model.predict(X_test[start:end+1])    
    
    result = []
    for cat, num in zip(Y_test[0], Y_test[1]):
        x_list = list(cat)
        label_name_x = label_name.copy()
        temp = []
        for col in cat_name:
            temp.append(label_name_x[x_list.index(max(x_list[0:len(all_data[col].unique())]))])
            
            # 출력한 Column과 데이터 삭제
            del x_list[:len(all_data[col].unique())]
            del label_name_x[:len(all_data[col].unique())]
        temp.extend(num)
        result.append(temp)
        
    
    return np.array(result)
    

In [17]:
# set result array to each cases
def setResult(arr, predict, case):
    result_arr = arr
    
    if case == [int(x) for x in range(0, 2)]:
        for row in range(len(predict)):
            result_arr[row, 2] = predict[row, 0] # 사망자수
            result_arr[row, 5] = predict[row, 1] # 사상자수            
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum() # 사상자수 = 사망자수 + (중상+경상+부상신고자수)
            
    elif case == [int(x) for x in range(2, 4)]:
        for row in range(len(predict)):
            result_arr[row, 4] = predict[row, 0]
            result_arr[row, 6] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(4, 7)]:
        for row in range(len(predict)):
            result_arr[row, 4] = predict[row, 0]
            result_arr[row, 5] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(7, 10)]:
        for row in range(len(predict)):
            result_arr[row, 2] = predict[row, 0]
            result_arr[row, 4] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(10,20)]:
        for row in range(len(predict)):
            result_arr[row, 9] = predict[row, 0] 
            result_arr[row, 10] = predict[row, 1] 
            result_arr[row, 11] = predict[row, 2] 
            print(result_arr[row, 11], predict[row, 2])
            
    elif case == [int(x) for x in range(20,23)]:
        for row in range(len(predict)):
            result_arr[row, 12] = predict[row, 0] 
            result_arr[row, 13] = predict[row, 1] 
            result_arr[row, 14] = predict[row, 2] 
            
    elif case == [int(x) for x in range(23,26)]:
        for row in range(len(predict)):
            result_arr[row, 12] = predict[row, 0] 
            result_arr[row, 13] = predict[row, 1] 
            result_arr[row, 15] = predict[row, 2] 
    
    elif case == [int(x) for x in range(26,30)]:
        for row in range(len(predict)):
            result_arr[row, 12] = predict[row, 0] 
            result_arr[row, 13] = predict[row, 1] 
            result_arr[row, 14] = predict[row, 2] 
            result_arr[row, 15] = predict[row, 3] 
    
    elif case == [int(x) for x in range(35,40)]:
        for row in range(len(predict)):
            result_arr[row, 7] = predict[row, 0] 
            result_arr[row, 8] = predict[row, 1] 
        
    elif case == [int(x) for x in range(40,42)]:
        for row in range(len(predict)):
            result_arr[row, 1] = predict[row, 0] 
            result_arr[row, 9] = predict[row, 1] 
            result_arr[row, 10] = predict[row, 2] 
        
    elif case == [int(x) for x in range(42,45)]:
        for row in range(len(predict)):
            result_arr[row, 1] = predict[row, 0] 
            result_arr[row, 10] = predict[row, 1] 
            result_arr[row, 11] = predict[row, 2] 
            result_arr[row, 12] = predict[row, 3] 
    
    elif case == [int(x) for x in range(30,32)]:
        for row in range(len(predict)):
            result_arr[row, 2] = float(predict[row, 1])
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            result_arr[row, 8] = predict[row, 0]
            
    elif case == [int(x) for x in range(32,35)]:
        for row in range(len(predict)):
            result_arr[row, 4] = predict[row, 1]
            result_arr[row, 5] = predict[row, 2]
            result_arr[row, 8] = predict[row, 0]
            
    elif case == [int(x) for x in range(45,47)]:
        for row in range(len(predict)):
            result_arr[row, 0] = predict[row, 0]
            result_arr[row, 2] = float(predict[row, 2])
            result_arr[row, 14] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
            
    elif case == [int(x) for x in range(47,50)]:
        for row in range(len(predict)):
            result_arr[row, 0] = predict[row, 0]
            result_arr[row, 4] = float(predict[row, 2])
            result_arr[row, 13] = predict[row, 1]
            result_arr[row, 3] = result_arr[row, 2] + result_arr[row, 4:7].sum()
    
    print(result_arr)
    return result_arr
    

In [18]:
x_test = pd.read_csv('./test_kor.csv',encoding='cp949')

# setResult(each case array, predict array, start to end in each case):

In [19]:
test_arr = np.array([x_test.columns.values])
print(test_arr)

[['주야' '요일' '사망자수' '사상자수' '중상자수' '경상자수' '부상신고자수' '발생지시도' '발생지시군구'
  '사고유형_대분류' '사고유형_중분류' '법규위반' '도로형태_대분류' '도로형태' '당사자종별_1당_대분류'
  '당사자종별_2당_대분류']]


In [20]:
# train, set result in new array
for (idx, case), case_range in zip(enumerate(Cases), test_ranges):
    # for matching case number +1 to idx
    print()
    # numeric case
    if (idx+1) <= 4:      
#         print(idx+1, case, case_range)
        Case_prediction = numeric_case(case, case_range[0], case_range[1])
        answer = setResult(x_test.loc[case_range[0]:case_range[1]].values, Case_prediction, [int(x) for x in range(case_range[0], case_range[1]+1)])
        test_arr = np.append(test_arr, answer, axis=0)
        
    # categorical case
    elif ((idx+1) >= 5 and (idx+1) <= 8) or ((idx+1) >= 11 and (idx+1) <= 13):
#         print(idx+1, case, case_range)
        Case_prediction = categorical_case(case, case_range[0], case_range[1])
        answer = setResult(x_test.loc[case_range[0]:case_range[1]].values, Case_prediction, [int(x) for x in range(case_range[0], case_range[1]+1)])
        test_arr = np.append(test_arr, answer, axis=0)

    # mixed case
    elif (idx+1) == 9 or (idx+1) == 10 or (idx+1) >= 14:
#         print(idx+1, case, case_range)
        
        if (idx+1) == 9 or (idx+1) == 10:
            num_categories = 1  # number of categories for each case
            
        if (idx+1) == 14 or (idx+1) == 15:
            num_categories = 2
            
        Case_prediction = mix_case(case, num_categories, case_range[0], case_range[1])
        answer = setResult(x_test.loc[case_range[0]:case_range[1]].values, Case_prediction, [int(x) for x in range(case_range[0], case_range[1]+1)])
        test_arr = np.append(test_arr, answer, axis=0)


Case: ['사망자수', '사상자수', '경상자수']
사상자제거: ['사망자수', '경상자수']
Train on 20029 samples, validate on 5008 samples
Epoch 1/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.8092 - acc: 0.9538 - val_loss: 0.7044 - val_acc: 0.9551
Epoch 2/50
20029/20029 [==============================] - 76s 4ms/step - loss: 0.6928 - acc: 0.9570 - val_loss: 0.7046 - val_acc: 0.9551
Epoch 3/50
20029/20029 [==============================] - 76s 4ms/step - loss: 0.6901 - acc: 0.9570 - val_loss: 0.7073 - val_acc: 0.9551
Epoch 4/50
20029/20029 [==============================] - 76s 4ms/step - loss: 0.6887 - acc: 0.9570 - val_loss: 0.7045 - val_acc: 0.9551
Epoch 5/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.6890 - acc: 0.9570 - val_loss: 0.7037 - val_acc: 0.9551
Epoch 6/50
20029/20029 [==============================] - 76s 4ms/step - loss: 0.6875 - acc: 0.9570 - val_loss: 0.6963 - val_acc: 0.9551
Epoch 7/50
20029/20029 [==============================] - 76s 4ms/step - 

20029/20029 [==============================] - 77s 4ms/step - loss: 0.6175 - acc: 0.9879 - val_loss: 1.0316 - val_acc: 0.9906
Epoch 22/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.6175 - acc: 0.9879 - val_loss: 1.0316 - val_acc: 0.9906
Epoch 23/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.6175 - acc: 0.9877 - val_loss: 1.0317 - val_acc: 0.9906
Epoch 24/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.6175 - acc: 0.9879 - val_loss: 1.0316 - val_acc: 0.9906
Epoch 25/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.6175 - acc: 0.9879 - val_loss: 1.0316 - val_acc: 0.9906
[['야간' '월' 1.0 1.3191795591264963 0.29236087 0.0 0.026818687 '전남' '곡성군'
  '차량단독' '전도전복' '안전운전 의무 불이행' '단일로' '기타단일로' '자전거' '없음']
 ['야간' '일' 2.0 3.3191795591264963 0.29236087 1.0 0.026818687 '대구' '달성군'
  '차대차' '측면충돌' '중앙선 침범' '단일로' '기타단일로' '승용차' '승합차']]

Case: ['사상자수', '중상자수', '경상자수']
사상자제거: ['중상자수', '경상자수']
Train on 20029

20029/20029 [==============================] - 76s 4ms/step - loss: 0.5390 - acc: 0.9337 - val_loss: 0.5682 - val_acc: 0.8726
Epoch 23/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.6340 - acc: 0.9303 - val_loss: 0.5911 - val_acc: 0.9381
Epoch 24/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.5866 - acc: 0.9453 - val_loss: 0.5909 - val_acc: 0.9381
Epoch 25/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.5873 - acc: 0.9453 - val_loss: 0.5953 - val_acc: 0.9381
Epoch 26/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.5867 - acc: 0.9453 - val_loss: 0.5899 - val_acc: 0.9381

Epoch 00026: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 27/50
20029/20029 [==============================] - 77s 4ms/step - loss: 0.5864 - acc: 0.9453 - val_loss: 0.5903 - val_acc: 0.9381
[['주간' '월' 1.0380177 6.3272318840026855 0.28921413 5.0 0.0 '충남' '서천군'
  '차대차' '추돌' '안전운전 의무 불이행' '단일로' 

20029/20029 [==============================] - 69s 3ms/step - loss: 0.1579 - acc: 0.9329 - val_loss: 0.1583 - val_acc: 0.9346
Epoch 26/50
20029/20029 [==============================] - 69s 3ms/step - loss: 0.1572 - acc: 0.9328 - val_loss: 0.1556 - val_acc: 0.9345
Epoch 27/50
20029/20029 [==============================] - 69s 3ms/step - loss: 0.1569 - acc: 0.9329 - val_loss: 0.1555 - val_acc: 0.9350
[['야간' '토' 1.0 11.0 0.0 0.0 10.0 '경기' '의왕시' '차대차' '추돌' '안전운전 의무 불이행'
  '단일로' '기타단일로' '승용차' '화물차']
 ['주간' '화' 1.0 13.0 3.0 9.0 0.0 '경기' '평택시' '차대차' '추돌' '안전운전 의무 불이행' '단일로'
  '기타단일로' '승용차' '화물차']
 ['주간' '화' 1.0 1.0 0.0 0.0 0.0 '전남' '광양시' '차대사람' '보도통행중' '안전운전 의무 불이행'
  '단일로' '기타단일로' '승용차' '보행자']]

Case: ['도로형태_대분류', '도로형태', '당사자종별_2당_대분류']
Train on 20029 samples, validate on 5008 samples
Epoch 1/50
20029/20029 [==============================] - 69s 3ms/step - loss: 0.1909 - acc: 0.9247 - val_loss: 0.1538 - val_acc: 0.9385
Epoch 2/50
20029/20029 [==============================] - 69s 3ms/step -

Epoch 2/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1696 - acc: 0.9308 - val_loss: 0.1638 - val_acc: 0.9330
Epoch 3/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1657 - acc: 0.9316 - val_loss: 0.1637 - val_acc: 0.9337
Epoch 4/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1643 - acc: 0.9322 - val_loss: 0.1618 - val_acc: 0.9334
Epoch 5/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1629 - acc: 0.9332 - val_loss: 0.1603 - val_acc: 0.9368
Epoch 6/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1588 - acc: 0.9365 - val_loss: 0.1534 - val_acc: 0.9379
Epoch 7/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1545 - acc: 0.9387 - val_loss: 0.1517 - val_acc: 0.9400
Epoch 8/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1497 - acc: 0.9412 - val_loss: 0.1417 - val_acc: 0.9444
Epoch 9/50
20029/20029 [=================

...

20029/20029 [==============================] - 26s 1ms/step - loss: 0.0434 - acc: 0.9911 - val_loss: 0.0437 - val_acc: 0.9911
Epoch 13/50
20029/20029 [==============================] - 26s 1ms/step - loss: 0.0434 - acc: 0.9911 - val_loss: 0.0434 - val_acc: 0.9911
Epoch 14/50
20029/20029 [==============================] - 26s 1ms/step - loss: 0.0434 - acc: 0.9911 - val_loss: 0.0432 - val_acc: 0.9911
Epoch 15/50
20029/20029 [==============================] - 26s 1ms/step - loss: 0.0433 - acc: 0.9911 - val_loss: 0.0431 - val_acc: 0.9911
Epoch 16/50
20029/20029 [==============================] - 26s 1ms/step - loss: 0.0433 - acc: 0.9911 - val_loss: 0.0431 - val_acc: 0.9911
Epoch 17/50
20029/20029 [==============================] - 26s 1ms/step - loss: 0.0433 - acc: 0.9911 - val_loss: 0.0432 - val_acc: 0.9911
Epoch 18/50
20029/20029 [==============================] - 26s 1ms/step - loss: 0.0433 - acc: 0.9911 - val_loss: 0.0431 - val_acc: 0.9911
Epoch 19/50
20029/20029 [=====================

Epoch 2/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1712 - acc: 0.9404 - val_loss: 0.1651 - val_acc: 0.9423
Epoch 3/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1681 - acc: 0.9407 - val_loss: 0.1642 - val_acc: 0.9423
Epoch 4/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1653 - acc: 0.9406 - val_loss: 0.1576 - val_acc: 0.9424
Epoch 5/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1592 - acc: 0.9408 - val_loss: 0.1549 - val_acc: 0.9424
Epoch 6/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1531 - acc: 0.9411 - val_loss: 0.1459 - val_acc: 0.9426
Epoch 7/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1525 - acc: 0.9413 - val_loss: 0.1450 - val_acc: 0.9430
Epoch 8/50
20029/20029 [==============================] - 65s 3ms/step - loss: 0.1483 - acc: 0.9414 - val_loss: 0.1425 - val_acc: 0.9428
Epoch 9/50
20029/20029 [=================

20029/20029 [==============================] - 1s 52us/step - loss: 0.3216 - cat_output_loss: 0.2471 - num_output_loss: 0.0745 - cat_output_acc: 0.8824 - num_output_acc: 0.9664 - val_loss: 0.3127 - val_cat_output_loss: 0.2471 - val_num_output_loss: 0.0656 - val_cat_output_acc: 0.8820 - val_num_output_acc: 0.9631
Epoch 5/50
20029/20029 [==============================] - 1s 52us/step - loss: 0.3141 - cat_output_loss: 0.2438 - num_output_loss: 0.0704 - cat_output_acc: 0.8837 - num_output_acc: 0.9668 - val_loss: 0.3266 - val_cat_output_loss: 0.2473 - val_num_output_loss: 0.0793 - val_cat_output_acc: 0.8829 - val_num_output_acc: 0.9627
Epoch 6/50
20029/20029 [==============================] - 1s 52us/step - loss: 0.3066 - cat_output_loss: 0.2408 - num_output_loss: 0.0658 - cat_output_acc: 0.8862 - num_output_acc: 0.9656 - val_loss: 0.3215 - val_cat_output_loss: 0.2460 - val_num_output_loss: 0.0755 - val_cat_output_acc: 0.8838 - val_num_output_acc: 0.9629
Epoch 7/50
20029/20029 [============

20029/20029 [==============================] - 1s 68us/step - loss: 1.2297 - cat_output_loss: 0.2402 - num_output_loss: 0.9895 - cat_output_acc: 0.8959 - num_output_acc: 0.7594 - val_loss: 1.1692 - val_cat_output_loss: 0.1818 - val_num_output_loss: 0.9874 - val_cat_output_acc: 0.9135 - val_num_output_acc: 0.8095
Epoch 2/50
20029/20029 [==============================] - 1s 52us/step - loss: 1.0599 - cat_output_loss: 0.1524 - num_output_loss: 0.9076 - cat_output_acc: 0.9231 - num_output_acc: 0.8008 - val_loss: 1.0839 - val_cat_output_loss: 0.1420 - val_num_output_loss: 0.9419 - val_cat_output_acc: 0.9273 - val_num_output_acc: 0.8139
Epoch 3/50
20029/20029 [==============================] - 1s 52us/step - loss: 1.0424 - cat_output_loss: 0.1422 - num_output_loss: 0.9003 - cat_output_acc: 0.9279 - num_output_acc: 0.8040 - val_loss: 1.0199 - val_cat_output_loss: 0.1378 - val_num_output_loss: 0.8821 - val_cat_output_acc: 0.9300 - val_num_output_acc: 0.7550
Epoch 4/50
20029/20029 [============

20029/20029 [==============================] - 1s 52us/step - loss: 0.7044 - cat_output_loss: 0.1226 - num_output_loss: 0.5818 - cat_output_acc: 0.9387 - num_output_acc: 0.8425 - val_loss: 1.0119 - val_cat_output_loss: 0.1212 - val_num_output_loss: 0.8907 - val_cat_output_acc: 0.9402 - val_num_output_acc: 0.7989
Epoch 27/50
20029/20029 [==============================] - 1s 52us/step - loss: 0.6226 - cat_output_loss: 0.1215 - num_output_loss: 0.5010 - cat_output_acc: 0.9392 - num_output_acc: 0.8447 - val_loss: 1.2059 - val_cat_output_loss: 0.1203 - val_num_output_loss: 1.0855 - val_cat_output_acc: 0.9401 - val_num_output_acc: 0.8025
Epoch 28/50
20029/20029 [==============================] - 1s 53us/step - loss: 0.5380 - cat_output_loss: 0.1209 - num_output_loss: 0.4171 - cat_output_acc: 0.9396 - num_output_acc: 0.8485 - val_loss: 1.0512 - val_cat_output_loss: 0.1202 - val_num_output_loss: 0.9310 - val_cat_output_acc: 0.9401 - val_num_output_acc: 0.7889
Epoch 29/50
20029/20029 [=========

In [21]:
test_arr

array([['주야', '요일', '사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수', '발생지시도',
        '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', '도로형태_대분류', '도로형태',
        '당사자종별_1당_대분류', '당사자종별_2당_대분류'],
       ['야간', '금', 1.0217822, 1.2552632838487625, 0.0, 0.23348112, 0.0,
        '경기', '화성시', '차대차', '측면충돌', '중앙선 침범', '단일로', '기타단일로', '승용차',
        '승합차'],
       ['야간', '금', 1.0217619, 1.2552275359630585, 0.0, 0.23346564, 0.0,
        '전남', '영암군', '차대사람', '차도통행중', '과속', '단일로', '기타단일로', '승용차', '보행자'],
       ['야간', '월', 1.0, 1.3191795591264963, 0.29236087, 0.0, 0.026818687,
        '전남', '곡성군', '차량단독', '전도전복', '안전운전 의무 불이행', '단일로', '기타단일로',
        '자전거', '없음'],
       ['야간', '일', 2.0, 3.3191795591264963, 0.29236087, 1.0, 0.026818687,
        '대구', '달성군', '차대차', '측면충돌', '중앙선 침범', '단일로', '기타단일로', '승용차',
        '승합차'],
       ['주간', '목', 1.0, 1.5583660006523132, 0.29901856, 0.25934744, 0.0,
        '전남', '고흥군', '차대차', '정면충돌', '중앙선 침범', '단일로', '기타단일로', '화물차',
        '화물차'],
       ['주간', '목', 2.0, 2.5583660

In [22]:
with open('./test_kor_1.csv','wb',) as f:
        np.savetxt(f,  test_arr,  delimiter=",", fmt='%s', encoding='cp949')

In [23]:
save_result()

Save Success
